# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts

### Setup and Initiate Spark

In [1]:
import re
import ast
import time
import itertools
import numpy as np
from numpy import allclose
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session (RUN THIS CELL AS IS)
from pyspark.sql import SparkSession
app_name = "hw3_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [5]:
# Spark configuration Information (RUN THIS CELL AS IS)
sc.getConf().getAll()

[('spark.app.id', 'local-1564709664252'),
 ('spark.app.name', 'hw3_notebook'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '39571'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'docker.w261')]

### Setup Path and read in raw data files

In [6]:
# Read the toy data file
toy_raw = ast.literal_eval(open("data/toy.txt", "r").read())

# Read the 10K sample data file for feature extraction and data preparation
tenK_raw = ast.literal_eval(open("data/eda.txt", "r").read())

In [8]:
# Read the 10K sample data file for feature extraction and data preparation
# f = open("data/eda.txt", "r")
# eda = ast.literal_eval(f.read())
# f.close()

# # parse each row and return in list form
# parsed_eda = sc.parallelize(eda).map(lambda x: x.split('\t')).collect()
# print("EDA row count:", len(parsed_eda))

### Parse Row Into Readable formats

In [7]:
def parse_raw_row(row):
    '''
    for each row in the raw data,  output is a list of label and all the features:
        - [label, feature_1, feature_1, ...]
    For first 13 features, change the data type to number.
    Remaining features will of type string.
    For null values, populate None
    '''
    row_values = row.split('\t')
    for i, value in enumerate(row_values):
        if i <14:
            row_values[i] = int(value) if value != '' else None
        else:
            row_values[i] = value if value != '' else "''"
    return row_values

In [8]:
# parse raw toy data to form toyRDD
toyRDD = sc.parallelize(toy_raw).map(parse_raw_row).cache()
#toyRDD.take(1)

# parse raw 10k sample data to form tenKRDD
tenKRDD = sc.parallelize(tenK_raw).map(parse_raw_row).cache()

## Create Dataframe for Feature Engineering ##

In [9]:
#### Create SQL dataframe from RDD

# for toy data
toyFeature_df = sqlContext.createDataFrame(toyRDD)

# for 10K sample data
tenKfeature_df = sqlContext.createDataFrame(tenKRDD)
#tenKfeature_df.dtypes

# tenKfeature_df = df.withColumnRenamed("_1", "label")
#                    .withColumnRenamed("_2", "f1")
#                    .withColumnRenamed("_2", "f2")
#                    .withColumnRenamed("_2", "f3")
#                    .withColumnRenamed("_2", "f4")
#                    .withColumnRenamed("_2", "f5")
#                 .withColumnRenamed("_2", "f6")
#                    .withColumnRenamed("_2", "f7")
#                     .withColumnRenamed("_2", "f8")
#                     .withColumnRenamed("_2", "f9")
#                     .withColumnRenamed("_2", "f10")
#                     .withColumnRenamed("_2", "f11")
#                     .withColumnRenamed("_2", "f12")
#                     .withColumnRenamed("_2", "f13")
#                     .withColumnRenamed("_2", "f14")
#                     .withColumnRenamed("_2", "f15")
#                     .withColumnRenamed("_2", "f16")
#                     .withColumnRenamed("_2", "f17")
#                     .withColumnRenamed("_2", "f18")
#                     .withColumnRenamed("_2", "f19")
#                      .withColumnRenamed("_2", "f20")
#                     .withColumnRenamed("_2", "f21")
#                      .withColumnRenamed("_2", "f22")
#                     .withColumnRenamed("_2", "f23")  
#                     .withColumnRenamed("_2", "f24")
#                     .withColumnRenamed("_2", "f25")
#                     .withColumnRenamed("_2", "f26")
#                     .withColumnRenamed("_2", "f27")
#                                         .withColumnRenamed("_2", "f28")
#                                          .withColumnRenamed("_2", "f29")
#                                              .withColumnRenamed("_2", "f30")
#                                          .withColumnRenamed("_2", "f31")
#                                     .withColumnRenamed("_2", "f32")
#                                         .withColumnRenamed("_2", "f33")
#                                          .withColumnRenamed("_2", "f34")
#                                                      .withColumnRenamed("_2", "f35")
#                                                     .withColumnRenamed("_2", "f36")                  
#                                                     .withColumnRenamed("_2", "f37")
#                                                              .withColumnRenamed("_39", "f38")
#                                          .withColumnRenamed("_40", "f39")
                                                                                                                                                    



#### Create Panda dataframe from RDD
# create pandas dataframe
# tenKfeature_df.toPandas() 

# features = ['label'] + list(range(1, 40))
# tenKpanda_df = pd.DataFrame(tenKRDD, columns = features)


## Feature Extraction

**1: Remove features with very large number of unknown data**  

From EDA we see that the below feature columns have more than 40% null values.  
Due to this high percenage of unknown data we won't keep these features in our model. So dropping these columns from our data frame.

In [10]:
# drop features with high unknown values

toy_df1 = toyFeature_df.drop('_13','_36','_2','_11','_33','_34','_39','_40')
tenK_df1 = tenKfeature_df.drop('_13','_36','_2','_11','_33','_34','_39','_40')

tenK_df1.show(1)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _1| _3| _4| _5| _6| _7| _8| _9|_10|_12|_14|     _15|     _16|     _17|     _18|     _19|     _20|     _21|     _22|     _23|     _24|     _25|     _26|     _27|     _28|     _29|     _30|     _31|     _32|     _35|     _37|     _38|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  0|  6|  3| 34|  4| 27|  3| 33| 34|  1| 15|05db9164|028bd518|77f2f2e5|d16679b9|25c83c98|fbad5c96|2ecb612f|5b392875|a73ee510|3b08e48b|4efc1873|9f32b866|f15f3681|b28479f6|9559bea6|31ca40b6|07c540c4|2a40f0da|dfcfc3fa|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+------

**2. Remove Categorical features with high % of Uniqueness of Categories**  

From EDA, we see that for the following categorical features uniqueness is more than 50%. When uniqueness of a feature is more than 50% it should not be having much impact on label prediction. So will remove those columns from our model.  
_17, _18, _21, _24, _26, _30, _35

In [11]:
toy_df2 = toy_df1.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df2 = tenK_df1.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df2.show(1)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _1| _3| _4| _5| _6| _7| _8| _9|_10|_12|_14|     _15|     _16|     _19|     _20|     _22|     _23|     _25|     _27|     _28|     _29|     _31|     _32|     _37|     _38|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  0|  6|  3| 34|  4| 27|  3| 33| 34|  1| 15|05db9164|028bd518|25c83c98|fbad5c96|5b392875|a73ee510|4efc1873|f15f3681|b28479f6|9559bea6|07c540c4|2a40f0da|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
only showing top 1 row



**3. Colenearity Reduction**  

EDA shows strong correlation between below numerical features:  
> _5 and _14  
> _5 and _9  
> _9 and _14  
> _8 and _12  
There is also a moderate negative correlation for feature:  
> _6 and _11  
> _7 and _11.  
To avoid co-leniarity we will remove feature _14, _9, _6, _7 and _8

In [116]:
toy_df3 = toy_df2.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df3 = tenK_df2.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df3.show(1)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _1| _3| _4| _5| _6| _7| _8| _9|_10|_12|_14|     _15|     _16|     _19|     _20|     _22|     _23|     _25|     _27|     _28|     _29|     _31|     _32|     _37|     _38|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  0|  6|  3| 34|  4| 27|  3| 33| 34|  1| 15|05db9164|028bd518|25c83c98|fbad5c96|5b392875|a73ee510|4efc1873|f15f3681|b28479f6|9559bea6|07c540c4|2a40f0da|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
only showing top 1 row



In [117]:
tenK_df3.cache()
tenK_df3.count()

10758

**4: Binning of Categories**  

From EDA we see that amongst remaining categorical features, unique category count for following  are very low.  
_23, _20    
For rest of the features we will try binning the categories into three distinct features. AT the same time converting the feature type to numerical.  
Binning is done by checking frequency% count.  
If frequency% > 50, value = 30  
50> frequesncy >20, value = 20  
Otherwise value = 10

For null values populating one of the bin value randomly

> Check frequency of unique categories of Categorical Features

In [172]:
## "_15" ##

# From EDA we see that this feature has 188 unique categories and no missing values
tenK_df3.groupBy("_15").count().show(2,False)

+--------+-----+
|_15     |count|
+--------+-----+
|7ceef477|5    |
|decf6fa6|2    |
+--------+-----+
only showing top 2 rows



Looking at the unique category distribution will bin the values in three categories with below logic:  

1. If value = '05db9164', new_value = 1  
2. If value in ('8cf07265', '68fd1e64', '5a9ed9b0'), new_value = 2  
3. For rest of the rows, new_value = 3  
At the same time we will convert the column type to numerical.

In [171]:
# To calculate click through rate frequency count of each category, we will drop the rows with zero clicks

#val output = df.where("col1>col2")

In [164]:
## "_16" ##

# From EDA we see that this feature has 406 unique categories and no missing values
#tenK_df3.cube("_16").count().show(406,False)

#dm = tenK_df3.groupBy("_31").count()
# dm1 = tenK_df3.groupBy("_15","_1").count()
# dm2 = tenK_df3.groupBy("_1").count()
#dm.show()

In [170]:
# F.array(
#     '10','20','30'
#   ).getItem(
#     (F.rand()*3)).cast("int")

In [168]:
####### Calculate frequency of unique category under each feature ########

#exclude the list of features that are already binned with just few categories
exclude_list = ['_23', '_20'] #, '_31'

count = 0
for i in tenK_df3.dtypes:
    if i[1]=='string' and i[0] not in exclude_list:
        feature = i[0]
        cat_freq = tenK_df3.cube(feature).count()
        cat_freqDF = cat_freq.withColumn('percent', ((col('count')/ 10758) * 100.0))
        df1 = cat_freqDF.alias('df1')
        if count== 0:
            df2 = tenK_df3.alias('df2')
        else:
            df2 = tenK_freq_df.alias('df2')
        
        tenK_freq_df = df1.join(df2, df1[feature] == df2[feature]).select('df2.*',
                                                                        df1['percent'].alias(feature+'%'))
        
        ###### Bin data into three categories based on the frequency percentage count.
        # if frequency % of a category is greather/ equal to 50%, assigned value =30
        # if frequency % of a category is greater than 19.99% but less than 50%, value = 20
        # for the rest value is 10
        tenK_freq_df = tenK_freq_df.withColumn(feature,
        F.when(((tenK_freq_df[feature+'%'] > 50)|(tenK_freq_df[feature+'%'] == 50))
           & (tenK_freq_df[feature] != ''),F.lit(30))
        .otherwise(
        F.when((tenK_freq_df[feature+'%'] < 50) & (tenK_freq_df[feature+'%'] > 19.99)
           & (tenK_freq_df[feature] != ''),F.lit(20))
        .otherwise(
        F.when(tenK_freq_df[feature] == '', F.array(F.lit(10),F.lit(20),F.lit(30)).getItem((F.rand()*3).cast("int")))
        .otherwise(F.lit(10)))))
        
        count += 1
    
    elif i[0] == '_23':
        feature = '_23'
        tenK_freq_df = tenK_freq_df.withColumn(feature,
        F.when(tenK_freq_df[feature] == 'a73ee510', F.lit(30))
        .otherwise(
        F.when(tenK_freq_df[feature] == '7cc72ec2', F.lit(20))
        .otherwise(
        F.when(tenK_freq_df[feature] == '', F.array(F.lit(10),F.lit(20),F.lit(30)).getItem((F.rand()*3).cast("int")))
        .otherwise(F.lit(10)))))
            
    elif i[0] == '_20':
        feature = '_20'    
        tenK_freq_df = tenK_freq_df.withColumn(feature,
        F.when(tenK_freq_df[feature] == '7e0ccccf', F.lit(30))
        .otherwise(
        F.when(tenK_freq_df[feature] == ('fbad5c96'), F.lit(20))
        .otherwise(
        F.when(tenK_freq_df[feature] == ('fe6b92e5'), F.lit(10))
        .otherwise(
        F.when(tenK_freq_df[feature] == '',
                F.array(F.lit(10),F.lit(20),F.lit(30),F.lit(5)).getItem((F.rand()*4).cast("int")))
        .otherwise(F.lit(5))))))

            
tenK_freq_df.cache()


DataFrame[_1: bigint, _3: bigint, _4: bigint, _5: bigint, _6: bigint, _7: bigint, _8: bigint, _9: bigint, _10: bigint, _12: bigint, _14: bigint, _15: int, _16: int, _19: int, _20: int, _22: int, _23: int, _25: int, _27: int, _28: int, _29: int, _31: int, _32: int, _37: int, _38: int, _15%: double, _16%: double, _19%: double, _22%: double, _25%: double, _27%: double, _28%: double, _29%: double, _31%: double, _32%: double, _37%: double, _38%: double]

In [178]:
tenK_df4 = tenK_freq_df.drop('_15%','_16%','_19%','_22%','_25%','_27%','_28%','_29%', '_31%', '_32%', '_37%', '_38%')
#tenK_df4.show()

**5. Replace null values in numerical variables with mean **

In [175]:
def fill_with_mean(this_df):

    '''
    replace None value in Numerical variables with mean value
    '''
    stats = this_df.agg(*(avg(c).alias(c) for i,c in enumerate(this_df.columns) if i<11 and c != '_1'))
    return this_df.na.fill(stats.first().asDict())


In [176]:
tenK_df5 = fill_with_mean(tenK_df4)

In [177]:
tenK_df5.show()

+---+---+----+---+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| _1| _3|  _4| _5|    _6| _7| _8| _9|_10|_12|_14|_15|_16|_19|_20|_22|_23|_25|_27|_28|_29|_31|_32|_37|_38|
+---+---+----+---+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  0|  6|   3| 34|     4| 27|  3| 33| 34|  1| 15| 30| 10| 30| 20| 10| 30| 10| 10| 20| 10| 10| 10| 20| 10|
|  0|521|   1|  2|   512| 21|  3| 18| 44|  1|  2| 10| 10| 30|  5| 30| 30| 10| 10| 20| 10| 20| 10| 10| 10|
|  1|  0|   1|  1|  4982|111|  0|  1| 10|  0|  1| 10| 10| 10| 20| 30| 30| 10| 10| 20| 10| 10| 10| 10| 10|
|  0|156|  75| 15|  1352|276|  2| 49|104|  1| 50| 30| 10| 30| 30| 10| 30| 10| 10| 20| 10| 10| 10| 20| 10|
|  1|  0|  12| 12|     1|  0| 16| 12| 81|  5|  0| 10| 10| 10|  5| 30| 30| 10| 10| 20| 10| 20| 10| 20| 10|
|  1|  0|  16|  7|  7263| 51|  5|  8|132|  3|  7| 30| 10| 30| 10| 30| 30| 10| 10| 20| 10| 20| 10| 10| 10|
|  0|  0|   1|  8|     0|  0|  2|  2|  1|  1| 

## This is rough version of RandomForest Classifier. Do not look at this ##########

In [26]:

toyFeature_df = sqlContext.createDataFrame(toyRDD)
toy_df = toyFeature_df.drop('_13','_36','_2','_11','_33','_34','_39','_40')
toyDF = fill_with_mean(toy_df)

**6: Run RandomForest ensemble to check featureImportances**:  
With remaining features we will run RandomForest ensemble classifier to check featureImportances matrices.  
We will extract the Features with higher featureImportances scores for our model.  


In [24]:


# one hot encoding and assembling

encoding_var = [i[0] for i in toyDF.dtypes if (i[1]=='string')]
num_var = [i[0] for i in toyDF.dtypes if (i[1]!='string') & (i[0]!= '_1')]
#encoding_var = [col for i, col in enumerate(toyDF.columns) if (i > 0) & (i<11)]
#num_var = [col for j, col in enumerate(toyDF.columns) if (j > 14)]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep')
                  for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c])
                  for c in encoding_var]
label_indexes = StringIndexer(inputCol = '_1', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = num_var + ['OHE_' + c for c in encoding_var]
                            , outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                             numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
#num_var

In [25]:
mod = pipe.fit(toyDF)

In [26]:
toyDF2 = mod.transform(toyDF)

In [27]:
mod.stages[-1].featureImportances

SparseVector(1182, {3: 0.0042, 6: 0.0272, 7: 0.0466, 10: 0.0033, 12: 0.01, 20: 0.021, 23: 0.009, 47: 0.0329, 53: 0.0145, 91: 0.013, 99: 0.0275, 114: 0.0278, 137: 0.0145, 142: 0.0066, 172: 0.0101, 200: 0.0119, 212: 0.0249, 213: 0.0145, 219: 0.0131, 258: 0.0198, 267: 0.0253, 272: 0.0194, 368: 0.0004, 389: 0.0355, 398: 0.0, 435: 0.009, 445: 0.0204, 450: 0.0124, 469: 0.0086, 476: 0.033, 493: 0.0, 574: 0.0229, 577: 0.023, 578: 0.0184, 598: 0.0198, 608: 0.0063, 719: 0.0203, 758: 0.0079, 789: 0.0088, 803: 0.0112, 833: 0.0086, 855: 0.0217, 866: 0.0257, 897: 0.0127, 903: 0.0227, 952: 0.0194, 962: 0.0139, 994: 0.0252, 1000: 0.0085, 1033: 0.0068, 1034: 0.0272, 1047: 0.0137, 1052: 0.0462, 1053: 0.0234, 1083: 0.0073, 1102: 0.0377, 1103: 0.0004, 1106: 0.0113, 1141: 0.0123})

In [28]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [29]:
ExtractFeatureImp(mod.stages[-1].featureImportances, toyDF2, "features")

,idx,name,score
7,7,_10,0.046568
1052,1052,OHE__35_0d3fb920,0.046160
1102,1102,OHE__37_32c7478e,0.037704
389,389,OHE__24_67eea4ef,0.035513
476,476,OHE__25_59256bd8,0.033034
47,47,OHE__16_2705da39,0.032902
114,114,OHE__17_73cee255,0.027846
99,99,OHE__17_3cc14b5b,0.027500
1034,1034,OHE__35_d8126a6f,0.027239
6,6,_9,0.027237


In [95]:
# df = spark.createDataFrame([
#      (1.0, Vectors.dense(1.0)),
#      (0.0, Vectors.sparse(1, [], []))], ["label", "features"])

# stringIndexer = StringIndexer(inputCol="_1", outputCol="indexed")
# si_model = stringIndexer.fit(toyFeature_df)
# td = si_model.transform(toyFeature_df)
# rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="indexed", seed=42)
# model = rf.fit(td)
# model.featureImportances
# SparseVector(1, {0: 1.0})

X = toyFeature_df.select("_2", "_3", "_4", "_5", "_6", "_7", "_8", "_9", "_10", "_11", "_12", "_13"
                         , "_14", "_15", "_16", "_17", "_18", "_19", "_20", "_21", "_22", "_23", "_23"
                         , "_24", "_25", "_26", "_27", "_28", "_29", "_30", "_31", "_32", "_33", "_34"
                         , "_35", "_36", "_37", "_38", "_39", "_40")
y = toyFeature_df.select("_1")

# clf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="indexed", seed=42)

# clf.fit(X, y)  
#print(clf.feature_importances_)

X.show(5)

+---+---+---+---+-----+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _2| _3| _4| _5|   _6| _7| _8| _9|_10|_11|_12|_13|_14|     _15|     _16|     _17|     _18|     _19|     _20|     _21|     _22|     _23|     _23|     _24|     _25|     _26|     _27|     _28|     _29|     _30|     _31|     _32|     _33|     _34|     _35|     _36|     _37|     _38|     _39|     _40|
+---+---+---+---+-----+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| ''| -1| ''| ''|23106| 90|  8| 32|103| ''|  2| ''| ''|68fd1e64|d833535f|b00d1501|d16679b9|25c83c98|fe6

IllegalArgumentException: 'Field "features" does not exist.\nAvailable fields: _1, _2, _3, _4, _5, _6, _7, _8, _9, _10, _11, _12, _13, _14, _15, _16, _17, _18, _19, _20, _21, _22, _23, _24, _25, _26, _27, _28, _29, _30, _31, _32, _33, _34, _35, _36, _37, _38, _39, _40, indexed'